Set-Up

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from scripts import request, download, extraction, stats, table
from scripts.constants import Constants

project_path = os.path.abspath(os.path.expandvars(Constants.PROJECT_PATH))
experiments_path = os.path.join(project_path, Constants.Directories.EXPERIMENTS)
extracted_tables_path = os.path.join(experiments_path, Constants.Directories.EXTRACTED_TABLE)

connection_info = request.extract_infos('../private.json')

Download Articles

In [ ]:
article_dir = "CS_Test"
articles_path = os.path.join(experiments_path, Constants.Directories.ARTICLES, article_dir)

json_file_name = f"{article_dir}.json"

In [ ]:
# Download articles as HTML files
search_query = '"Entity Matching"'
download.get_articles(search_query, articles_path)

In [ ]:
extraction.extract_and_save_tables(articles_path, extracted_tables_path, json_file_name)

Claim Extraction

In [ ]:
output_dir = "cs_test"
tables_file_name = 'CS_Test.json'
msgs_dir = "CS_Outcome"

output_path = os.path.join(experiments_path, Constants.Directories.OUTPUT, output_dir)
tables_file_path = os.path.join(extracted_tables_path, tables_file_name)
msgs_path = os.path.join(project_path, Constants.Directories.MESSAGES, msgs_dir)

In [ ]:
request.set_up_test_dir(output_path, tables_file_path, msgs_path)
test_info = request.get_test_info(output_path)

In [ ]:
request.run_test(connection_info, test_info, num_thread=50, max_cycles=3)

Ground Truth

In [ ]:
whole_dataset_path = os.path.join(extracted_tables_path, "CS_Dataset.json")
extraction.check_extracted_data(whole_dataset_path)

In [ ]:
gt_path = os.path.join(experiments_path, Constants.Directories.GROUND_TRUTH)
gt_file = os.path.join(gt_path, "table_labeled.ods")
gt_answer_path = os.path.join(gt_path, "labeling")

stats.write_ground_truth(gt_file, gt_answer_path)

In [ ]:
output_dir = os.path.join(experiments_path, Constants.Directories.OUTPUT)
caption_dir = os.path.join(output_dir, "cs_gt_caption")
html_dir = os.path.join(output_dir, "cs_gt_html")

output_dirs = [
    (caption_dir, 1),
    (html_dir, 1)
]

json_path = os.path.join(experiments_path, "comparison.json")
stats.compare_results(gt_answer_path, output_dirs, json_path)